# Assembling the imperfect waterbomb elements

Show the structure of implementation with distributed responsibilities:

**WBCell:** individual cell related operations.

**WBAssembly:** responsible for assembling the cells. It can use classes like CellCombinator and CellEvaluator to get the job done.

**WBCombinator:** take care of generating all possible permutations of cells.

In [ ]:
import k3d
import numpy as np
import matplotlib.pylab as plt
from bmcs_shell.folding.assembly.wb_assembler import WBAssembler

# Modules 3

### Read the scanned files for individual cells

Demonstrator series

In [ ]:
modules = {'WB305': ('WB305_facets_points.obj', [[1, 0], [np.pi/2, np.pi/2]]),
           'WB306': ('WB306_facets_points.obj', [[1, 0], [np.pi/2, -np.pi/2]]),
           'WB307': ('WB307_facets_points.obj', [[1], [np.pi/2]]),
           'WB308': ('WB308_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
           'WB309': ('WB309_facets_points.obj', [[0, 2], [np.pi/2, -np.pi/2]]),
           'WB310': ('WB310_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
           'WB311': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
           'WB312': ('WB312_facets_points.obj', [[1], [-np.pi/2]]),
           'WB313': ('WB307_facets_points.obj', [[1], [np.pi/2]]),
           'WB314': ('WB310_facets_points.obj', [[1, 2], [np.pi/2, np.pi]]),
           'WB315': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
           'WB316': ('WB311_facets_points.obj', [[1], [np.pi/2]]),
           }

assembly = WBAssembler(modules=modules)

### Check the orientation of the individual cells

To ensure a link to the real cells check to see that the cell is defined
in the $x-y$ plane and its marker is on the bottom right corner.

In [ ]:
wb = assembly.wbs[4]
plot = k3d.plot(name='Vectors')
wb.plot_groups_of_points(plot, wb.wb_scan_X_Fia)
wb.plot_points(plot, wb.centroids_Fa, point_size=20, 
                      color=0x000000, plot_numbers=True)
plot.display()

### Show geometrical deviations

Plot all cells in a local coordinate system located in the center of the cell one over the other to see the amount of the deviations between individual cells.

In [ ]:

plot = k3d.plot(name='Vectors')
#wb.plot_points(plot, wb.O_centroids_Fa, plot_numbers=True)
assembly.plot_modules_3D(plot, facet_numbers=False, module_numbers=False)
plot.display()

### Apply the assembly scenario

The position of the cells is defined by running the assembly sequence. Due to imperfections, each plug step means a deviation from the ideal geometry. To avoid cumulation of imprecisions, the order and orientation of cells will be varied with the goal to compensate for the imprecision within such that the misfit between cells is minimized.

In [ ]:
plot = k3d.plot(name='Vectors')
assembly.plug_modules()
assembly.plot_modules_3D(plot)
plot.display()

### Verify the contact conditions

Show the contact points of two cells in local coordinate system. Calculation of rotation
and translation in horizontal and vertical directions requires that the marked points are 
lie on the plane of the facet that is indicated by the indicated normal vector.

In [ ]:
%matplotlib widget
wb_C = assembly.wbs[8]
wb_D = assembly.wbs[8]
fig, ax = plt.subplots(1,1, figsize=(8,2))
wb_C.plot_plugged_neighbors_yz(wb_D, diag_dir=1, y_dir=1, ax=ax)

### Plot the longitudinal view of the assembly

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,2))
assembly.plot_modules_yz(ax)

###  Plot transverse view of the assembly

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,2))
assembly.plot_modules_xz(ax)

### Quantification of misfit

In inter-module contacts which were not included in the plugin scenario, quantify the gaps or penetrations as a basis for minimization of the misfit. 

In [ ]:
assembly.get_module_misfits()

## Identify the best assembly

### All possible cell arrangements

Combine 8 entities of type A and four entities of type B in all possible ordered sequences of 12 entities. While the type A entities go to the first 8 slots, the type B entities go to the last 4 slots. The entities are identified by integers starting from 1 ... 8 for type A and 9 ... 12 for type B. Further, the type A entities are assumed to have either positive or negative sign. However, their combinations are not assumed arbitrary. The signs in the slots [0, 1, 3, 4, 6, 7] are assumed always opposite to the slots [2, 5]. 

In [ ]:
assembly.all_perms

### Evaluate misfit for random combinations

If only whole cells are subject to the permutation, alternating orientation is prescribed, and the boundary cells are not permuted, the total number combinations is 1.9 millions.

To reduce the computation time for debugging, the random choice from all combinations is used to find assembly arrangement with low amount of misfit.

In [ ]:
assembly.n_samples = 100

### Choose the best assembly and plot it

In [ ]:
assembly.best_combination_index

### Show the misfit of the best combination obtained

In [ ]:
assembly.misfit_of_best_combination

In [ ]:
plot = k3d.plot(name='Vectors')
assembly.plot_best_combination_3D(plot)
plot.display()

In [ ]:
fig, ax = plt.subplots(figsize=(8,2))
fig.canvas.header_visible=False
assembly.plot_best_combination_yz(ax)

## Extract the plugin support heights

In [ ]:
assembly.activate_best_combination
assembly.cell_enum[assembly.module_x_rows[0]]
assembly.module_x_rows


In [ ]:
assembly.get_support_geometry()

Plot a vertical column of a width W and height H which denotes the distance from the floor at the level zero to a saddle point in the located at the height H in the middle of the column. There are two lines starting from the saddle towards the vertical boundaries of the column symmetrically at an angle $\gamma < \pi/2$. The intersections of these two lines with the vertical boundaries of the columns are higher than the saddle point. The column is filled with color `c`. The upper boundary of the column has a V-shape. An additional feature of the function is the possibility to tilt the V-shape by an angle $\alpha < \pi / 4$.    
 

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
fig.canvas.header_visible=False
assembly.plot_longitudinal_supports(ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
fig.canvas.header_visible=False
assembly.plot_transverse_supports(ax)
ax.set_axis_on()

### Get the average angle of the middle opposite facets [3, 10] 

In [ ]:
assembly.gamma_support

In [ ]:
assembly.get_bounding_box()